In [44]:
from pathlib import Path
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os,random
import numpy as np
import augmentation
import collections
import warnings
warnings.simplefilter('ignore')

In [2]:
path=Path("genres")
genre_class=[]
audio_data=[]
cat_2_num={}
numeric_category=-1

for folder in path.glob("*"):
    numeric_category+=1
    category=str(folder).split("/")[-1]
    cat_2_num[category]=numeric_category

    for file in folder.glob("*.au"):
        data=str(file).split("/")[-1]
        audio_data.append(data)
        genre_class.append(cat_2_num[category])


In [3]:
#split datset
zip_list=list(zip(audio_data,genre_class))
random.seed(8)
random.shuffle(zip_list)

train_set=zip_list[:int(0.8*(len(zip_list)))]
test_set=zip_list[int(0.8*(len(zip_list))):int(0.9*(len(zip_list)))]
val_set=zip_list[int(0.9*(len(zip_list))):]


In [4]:
def audio_clip(clip):
    folder_name=clip[0].split(".")[0]
    file_path=path/folder_name/clip[0]
    audio_file,sr=librosa.load(file_path)#load file as time series in numpy array.sampling rate=22050
    return audio_file,folder_name

In [22]:
def create_images(data,target_folder):
    #save training set images into the target directory
    for clip in train_set:
        audio_file=audio_clip(clip)
        if target_folder=="train set":
            audio_file=augmentation.augmentation(audio_file)
            mel=list(map(librosa.feature.melspectrogram,audio_file))#compute melspectrogram
            for i in range(len(mel)):
                librosa.display.specshow(librosa.power_to_db(mel[i],ref=np.max),y_axis="off",x_axis="off")#plot melspectrogram
        else:
            mel=librosa.feature.melspectrogram(audio_file)
        
        #save melspectrograms in target folders
        image_name=os.path.splitext(clip[0])[0]
        dir_path=Path(target_folder)/folder_name
        dir_path.mkdir(parents=True,exist_ok=True)
        plt.axis("off")
        #plt.gca().xaxis.set_major_locator(plt.NullLocator())
        #plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.savefig(dir_path/(image_name+str(i)+".jpg"),bbox_inches='tight',pad_inches = 0)
        plt.close()

create_images(train_set,"train set")
create_images(val_set,"val set")
create_images(test_set,"test set")        

In [5]:
from librosa.feature import zero_crossing_rate,rmse,mfcc,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,spectral_contrast
from librosa.beat import tempo

In [73]:
#extract features for machine learning models
def create_features(audio):
    #time domain features
    zcr=zero_crossing_rate(audio)#array of fractions of zero crossing of each frame
    rms=rmse(audio)#array of rmse of each frame
    tempo_=tempo(audio)#beats per minute (scaler value)
    
    #frequency domain features
    mfcc_=mfcc(audio,n_mfcc=20)#computes mel-freq cepstral coefficients
    chroma=chroma_stft(audio)#computes chroma bins(12) for each frame
    spec_cent=spectral_centroid(audio)#centroid frequencies
    spec_band=spectral_bandwidth(audio,p=2)#pth order moment about spectral centroid
    spec_cont=spectral_contrast(audio)#min. max. difference between frequency bands
    spec_rolloff=spectral_rolloff(audio)#roll off frequency
    
    #take mean of each feature across frames
    features=[zcr,rms,spec_cent,spec_band,spec_cont,spec_rolloff,mfcc_,chroma]
    features=list(map(lambda x : np.mean(x,axis=1),features))
    features=[tempo_]+features
    
    return features

In [78]:
def extract_features(dataset):
    temp=[]
    for clip in dataset:
        audio,label=audio_clip(clip)
        feature=create_features(audio)
        flat_list=[value for sublist in feature for value in sublist]#flatten the extracted list    
        temp.append([label]+flat_list)
        if len(temp)%100==0:
            print(len(temp))
    return temp   

In [79]:
#create column list for dataframe
column=["label","tempo_","zcr","rms","spec_cent","spec_band","spec_cont","spec_rolloff"]
mfcc_col=["mfcc"+str(i) for i in range(20)]
chroma_col=["chroma"+str(i) for i in range(12)]
column.extend(mfcc_col+chroma_col)

#extract features of all examples
features=extract_features(zip_list)


KeyboardInterrupt: 